can run but the opening and close price is not correct. might be the timezone reason


In [ ]:
from ib_insync import *
import pandas as pd
import datetime
import pytz

# Connect to the Interactive Brokers API
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)

# Define the contract for NVDA
contract = Stock('NVDA', 'SMART', 'USD')

# Define the date for backtesting in Eastern Time
backtesting_date_et = datetime.datetime(2024, 3, 15, tzinfo=pytz.timezone('US/Eastern'))
market_open_time_et = backtesting_date_et.replace(hour=9, minute=30)  # Market opens at 9:30 AM ET
market_close_time_et = backtesting_date_et.replace(hour=16, minute=0)  # Market closes at 4:00 PM ET

# Convert times to strings with time zone information for the API
market_open_time_str = market_open_time_et.strftime('%Y%m%d %H:%M:%S') + ' US/Eastern'
market_close_time_str = market_close_time_et.strftime('%Y%m%d %H:%M:%S') + ' US/Eastern'

# Fetch the whole day's price data in 1-minute intervals
def get_full_day_prices(start_time_str, end_time_str):
    duration = (market_close_time_et - market_open_time_et).seconds
    bars = ib.reqHistoricalData(
        contract,
        endDateTime=end_time_str,
        durationStr=f'{duration} S',  # Duration in seconds to cover the whole trading day
        barSizeSetting='1 min',  # 1-minute bars
        whatToShow='TRADES',
        useRTH=True,  # Use regular trading hours
        formatDate=1  # Readable date format
    )
    return bars

# Get the historical data for the entire trading day
full_day_bars = get_full_day_prices(market_open_time_str, market_close_time_str)

# Convert the data into a DataFrame
# Convert the data into a DataFrame
if full_day_bars:
    df = pd.DataFrame([{
        'time': bar.date.astimezone(pytz.timezone('UTC')).replace(tzinfo=None),  # Convert to UTC and remove timezone
        'open': bar.open,
        'high': bar.high,
        'low': bar.low,
        'close': bar.close,
        'volume': bar.volume
    } for bar in full_day_bars])

    # Save the DataFrame to an Excel file
    excel_file = '/Users/yf/Documents/Trading/Trading/nvda_prices.xlsx'
    try:
        df.to_excel(excel_file, index=False)
        print(f"Data saved to {excel_file}")
    except Exception as e:
        print(f"Error saving Excel file: {e}")
else:
    print("No data received. Please check your request parameters.")


# Disconnect from IB
ib.disconnect()
